# 7.4 항공 사진 내 선인장 식별 경진대회 모델 성능 개선
- [항공 사진 내 선인장 식별 경진대회 링크](https://www.kaggle.com/c/aerial-cactus-identification)
- [모델링 노트북 참고 링크](https://www.kaggle.com/bonhart/simple-cnn-on-pytorch-for-beginers)


In [1]:
import pandas as pd

# 데이터 경로
data_path = '/kaggle/input/aerial-cactus-identification/'

labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [2]:
from zipfile import ZipFile

# 훈련 이미지 데이터 압축 풀기
with ZipFile(data_path + 'train.zip') as zipper:
    zipper.extractall()
    
# 테스트 이미지 데이터 압 풀기
with ZipFile(data_path + 'test.zip') as zipper:
    zipper.extractall()

In [3]:
import torch # 파이토치 
import random
import numpy as np
import os

# 시드 값 고정
seed = 10

os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
_, valid = train_test_split(labels, 
                            test_size=0.1,
                            stratify=labels['has_cactus'],
                            random_state=10)

In [6]:
import cv2
from torch.utils.data import Dataset # 데이터 생성을 위한 클래스

class ImageDataset(Dataset):
    # 초기화 메서드(생성자)
    def __init__(self, df, img_dir = './', transform=None):
        super().__init__() # 상속받은 Dataset의 __init__() 메서드 호출 
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
    
    # 데이터 세트 크기 반환 메서드 
    def __len__(self):
        return len(self.df)
    
    # 인덱스(idx)에 해당하는 데이터 반환 메서드 
    def __getitem__(self, idx):
        img_path = self.img_dir + self.df.iloc[idx, 0] # 이미지 파일 경로 
        image = cv2.imread(img_path) # 이미지 파일 읽기 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 이미지 색상 보정
        label = self.df.iloc[idx, 1] # 이미지 레이블(타깃 값)
        # 이미지 변환
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [7]:
from torchvision import transforms # 이미지 변환을 위한 모듈

# 훈련 데이터용 변환기
transform_train = transforms.Compose([transforms.ToPILImage(),
                                      transforms.Pad(32, padding_mode='symmetric'),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.RandomRotation(10),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                           std=[0.2, 0.2, 0.2])])
# 검증 및 테스트 데이터용 변환기
transform_test= transforms.Compose([transforms.ToPILImage(),
                                    transforms.Pad(32, padding_mode='symmetric'),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                         std=[0.2, 0.2, 0.2])])

In [8]:
dataset_train = ImageDataset(df=labels, img_dir='train/', transform=transform_train)
dataset_valid = ImageDataset(df=valid, img_dir='train/', transform=transform_test)

In [9]:
from torch.utils.data import DataLoader # 데이터 로더 생성을 위한 클래스

loader_train = DataLoader(dataset=dataset_train, batch_size=32, 
                          shuffle=True)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=32, 
                          shuffle=False)

In [10]:
import torch.nn as nn # 신경망 모듈
import torch.nn.functional as F # 신경망 모듈에서 자주 사용되는 함수

class Model(nn.Module):
    # 신경망 계층 정의
    def __init__(self):
        super().__init__() # 상속받은 nn.Module의 __init__() 메서드 호출
        # 첫 번째 ~ 다섯 번째 합성곱, 배치 정규화, 최대 풀링 계층 
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=32,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(32), # 배치 정규화 ---①
                                    nn.LeakyReLU(), # LeakyReLU 활성화 함수 ---②
                                    nn.MaxPool2d(kernel_size=2))

        self.layer2 = nn.Sequential(nn.Conv2d(in_channels=32, out_channels=64,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(64),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        
        self.layer3 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=128,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(128),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        
        self.layer4 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=256,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(256),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        
        self.layer5 = nn.Sequential(nn.Conv2d(in_channels=256, out_channels=512,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(512),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        # 평균 풀링 계층 
        self.avg_pool = nn.AvgPool2d(kernel_size=4) 
        # 전결합 계층 ---③
        self.fc1 = nn.Linear(in_features=512 * 1 * 1, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=2)

    # 순전파 출력 정의 
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.avg_pool(x)
        x = x.view(-1, 512 * 1 * 1) # 평탄화
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [11]:
model = Model().to(device)

model

Model(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Sequential(
    (0): Conv2d(128,

In [12]:
# 손실함수
criterion = nn.CrossEntropyLoss()

In [13]:
# 옵티마이저
optimizer = torch.optim.Adamax(model.parameters(), lr=0.0001)

In [14]:
epochs = 50 # 총 에폭

# 총 에폭만큼 훈련
for epoch in range(epochs):
    epoch_loss = 0 # 에폭별 손실값 초기화
    # 미니배치 크기만큼 데이터를 추출하는 작업을 '반복 횟수'만큼 반복 
    for images, labels in loader_train:
        # 이미지, 레이블 데이터 미니배치를 장비에 할당 
        images = images.to(device)
        labels = labels.to(device)
        
        # 옵티마이저 내 기울기 초기화
        optimizer.zero_grad()
        # 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 손실함수를 활용해 outputs와 labels의 손실값 계산
        loss = criterion(outputs, labels)
        # 역전파 수행
        loss.backward()
        # 가중치 갱신
        optimizer.step()
        
        epoch_loss += loss.item() # 현재 배치에서의 손실 추가
        
    print(f'에폭: [{epoch+1}/{epochs}], 손실값: {epoch_loss/len(loader_train)}')
#     print(f'에폭: [{epoch+1}/{epochs}], 손실값: {epoch_loss/len(loader_train):.4f}')    

에폭: [1/50], 손실값: 0.1086574530755076
에폭: [2/50], 손실값: 0.05804724411032685
에폭: [3/50], 손실값: 0.043110862700779407
에폭: [4/50], 손실값: 0.03675261308081317
에폭: [5/50], 손실값: 0.030205570332507196
에폭: [6/50], 손실값: 0.02575980902030567
에폭: [7/50], 손실값: 0.022373778547458388
에폭: [8/50], 손실값: 0.0209674963421817
에폭: [9/50], 손실값: 0.019303887756216814
에폭: [10/50], 손실값: 0.01934895049176184
에폭: [11/50], 손실값: 0.017283474925955056
에폭: [12/50], 손실값: 0.016964421172612643
에폭: [13/50], 손실값: 0.014153039533096304
에폭: [14/50], 손실값: 0.015826131744056785
에폭: [15/50], 손실값: 0.014254071956306846
에폭: [16/50], 손실값: 0.011525362329093713
에폭: [17/50], 손실값: 0.011935787669279663
에폭: [18/50], 손실값: 0.01256652732749159
에폭: [19/50], 손실값: 0.009528253359871123
에폭: [20/50], 손실값: 0.009387395167918365
에폭: [21/50], 손실값: 0.01006486846292495
에폭: [22/50], 손실값: 0.009579915464773693
에폭: [23/50], 손실값: 0.010873781835885174
에폭: [24/50], 손실값: 0.008641142771886379
에폭: [25/50], 손실값: 0.007643915784983603
에폭: [26/50], 손실값: 0.007838625616110699
에폭: [

In [15]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수

# 예측 확률값과 실제값을 담을 리스트 초기화
true_list = []
preds_list = []

model.eval() # 모델을 평가 상태로 설정 

with torch.no_grad(): # 기울기 계산 비활성 -
    for images, labels in loader_valid:
        # 이미지, 레이블 데이터 미니배치를 장비에 할당 
        images = images.to(device)
        labels = labels.to(device)
        # 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        preds = torch.softmax(outputs.cpu(), dim=1)[:, 1] # 예측 확률값 
        true = labels.cpu() # 실제값 
        # 예측 확률값과 실제값을 리스트에 추가
        preds_list.extend(preds)
        true_list.extend(true)
        
    # 검증 데이터 ROC AUC 점수 계산 
    print(f'검증 데이터 ROC AUC : {roc_auc_score(true_list, preds_list)}')
#     print(f'검증 데이터 ROC AUC : {roc_auc_score(true_list, preds_list):.4f}')    

검증 데이터 ROC AUC : 1.0


In [16]:
dataset_test = ImageDataset(df=submission, img_dir='test/', transform=transform_test)
loader_test = DataLoader(dataset=dataset_test, batch_size=32, 
                         shuffle=False)

In [17]:
model.eval() # 모델을 평가 상태로 설정

preds = [] # 타깃 예측 값 저장용 변수 초기화

with torch.no_grad(): # 기울기 계산 비활성
    for images, _ in loader_test:
        # 이미지 데이터 미니배치를 장비에 할당
        images = images.to(device)
        # 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 타깃 값이 1일 확률(예측 값)
        preds_part = torch.softmax(outputs.cpu(), dim=1)[:, 1].tolist()
        # preds에 preds_part 이어붙이기
        preds.extend(preds_part)

In [18]:
submission['has_cactus'] = preds
submission.to_csv('submission.csv', index=False)

In [19]:
submission

,id,has_cactus
0,000940378805c44108d287872b2f04ce.jpg,1.000000
1,0017242f54ececa4512b4d7937d1e21e.jpg,1.000000
2,001ee6d8564003107853118ab87df407.jpg,0.000003
3,002e175c3c1e060769475f52182583d0.jpg,0.000048
4,0036e44a7e8f7218e9bc7bf8137e4943.jpg,0.384290
...,...,...
3995,ffaafd0c9f2f0e73172848463bc2e523.jpg,1.000000
3996,ffae37344310a1549162493237d25d3f.jpg,1.000000
3997,ffbd469c56873d064326204aac546e0d.jpg,1.000000
3998,ffcb76b7d47f29ece11c751e5f763f52.jpg,1.000000


In [20]:
import shutil

shutil.rmtree('./train')
shutil.rmtree('./test')